In [ ]:
import requests
import json

# URL pointing directly to the raw JSON data
url = 'https://huggingface.co/datasets/BoKelvin/SLAKE/raw/main/train.json'

# Sending a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    try:
        # Load the JSON data
        data = response.json()
        
        # Specify the filename
        filename = 'train.json'
        
        # Write JSON data to a file
        with open(filename, 'w') as file:
            json.dump(data, file, indent=4)
        
        print(f"Data downloaded and saved successfully to {filename}!")
        
    except ValueError as e:
        print("Failed to decode JSON: ", e)
else:
    print("Failed to download data. Status code:", response.status_code)


Data downloaded and saved successfully to train.json!


In [ ]:
import json

def find_keys(obj, keys_set):
    """ Recursively find all keys in a JSON object and add them to a set. """
    if isinstance(obj, dict):
        for key, value in obj.items():
            keys_set.add(key)
            find_keys(value, keys_set)
    elif isinstance(obj, list):
        for item in obj:
            find_keys(item, keys_set)

def main():
    filename = 'train.json'  # Path to your JSON file

    # Load the JSON data from the file
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
        
        # Set to store unique keys
        unique_keys = set()
        
        # Find all unique keys in the JSON data
        find_keys(data, unique_keys)
        
        # Print all unique keys
        print("Unique keys in the JSON file:")
        for key in unique_keys:
            print(key)

    except FileNotFoundError:
        print(f"Error: The file {filename} does not exist.")
    except json.JSONDecodeError:
        print("Error: Failed to decode JSON from the file.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()



Unique keys in the JSON file:
content_type
modality
question
base_type
triple
q_lang
answer
location
qid
answer_type
img_id
img_name


In [4]:
import os
import requests

# Define the URL and the download path
url = 'https://huggingface.co/datasets/BoKelvin/SLAKE/raw/main/imgs.zip'
download_path = '/Users/jeffreysherer/Downloads/BBandMaskProcessing/imgs.zip'

# Create directory if it does not exist
os.makedirs(os.path.dirname(download_path), exist_ok=True)

try:
    # Download the file
    response = requests.get(url, stream=True, timeout=10)
    response.raise_for_status()

    # Write to file
    with open(download_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)

    print(f"File downloaded successfully: {download_path}")

except requests.exceptions.RequestException as e:
    print(f"Download failed: {e}")
except FileNotFoundError as e:
    print(f"File not found error: {e}")
except Exception as e:
    print(f"An error occurred: {e}")


File downloaded successfully: /Users/jeffreysherer/Downloads/BBandMaskProcessing/imgs.zip


In [6]:
import zipfile

zip_path = '/teamspace/studios/this_studio/imgs.zip'

# Check if the file is a valid zip file
is_zip = zipfile.is_zipfile(zip_path)
if is_zip:
    print(f"{zip_path} is a valid zip file.")
else:
    print(f"{zip_path} is not a valid zip file or is corrupted.")


/teamspace/studios/this_studio/imgs.zip is not a valid zip file or is corrupted.


In [23]:
import os
import shutil

# Base directory containing source folders
source_base_dir = '/teamspace/studios/this_studio/imgs'

# Target base directory to store subfolders with JSON files
target_base_dir = '/teamspace/studios/this_studio/processed_jsons'

# Ensure the target directory exists
if not os.path.exists(target_base_dir):
    os.makedirs(target_base_dir)

# Check if the source directory exists
if not os.path.exists(source_base_dir):
    print(f"Error: Source directory {source_base_dir} does not exist.")
else:
    # Loop through each directory in the source base directory
    for dirname in sorted(os.listdir(source_base_dir)):
        if dirname.startswith('xmlab'):  # Check if the directory name starts with 'xmlab'
            source_dir_path = os.path.join(source_base_dir, dirname)
            json_file_path = os.path.join(source_dir_path, 'detection.json')

            if os.path.isfile(json_file_path):  # Check if the JSON file exists
                # Create a new subdirectory for this JSON file in the target directory
                new_subdir_name = f"{dirname}detection"
                target_subdir_path = os.path.join(target_base_dir, new_subdir_name)
                if not os.path.exists(target_subdir_path):
                    os.makedirs(target_subdir_path)

                # Define the target file path
                target_file_path = os.path.join(target_subdir_path, 'detection.json')

                # Copy the JSON file to the new location
                shutil.copy2(json_file_path, target_file_path)
                print(f"File {json_file_path} copied to {target_file_path}")
            else:
                print(f"No detection.json found in {source_dir_path}")
        else:
            print(f"Skipped {dirname} as it does not match the 'xmlab*' pattern")


Error: Source directory /teamspace/studios/this_studio/imgs does not exist.


In [ ]:
import os
import json
import shutil

def has_numerical_values(data):
    """Recursively search for any numerical value in JSON data."""
    if isinstance(data, dict):
        return any(has_numerical_values(v) for v in data.values())
    elif isinstance(data, list):
        return any(has_numerical_values(item) for item in data)
    elif isinstance(data, (int, float)):
        return True
    return False

# Base directory containing the processed JSON folders
base_dir = '/teamspace/studios/this_studio/processed_jsons'
# New base directory for only numerical detections
numerical_base_dir = '/teamspace/studios/this_studio/numerical_detections'

# Ensure the target directory exists
if not os.path.exists(numerical_base_dir):
    os.makedirs(numerical_base_dir)

# Loop through each subdirectory in the base directory
for subdir in os.listdir(base_dir):
    subdir_path = os.path.join(base_dir, subdir)
    json_file_path = os.path.join(subdir_path, 'detection.json')

    if os.path.isfile(json_file_path):
        # Read the JSON file
        with open(json_file_path, 'r') as file:
            try:
                data = json.load(file)
            except json.JSONDecodeError:
                print(f"Failed to decode JSON in file {json_file_path}")
                continue

        # Check if the JSON data contains any numerical values
        if has_numerical_values(data):
            # Prepare a similar directory structure in the numerical detections directory
            numerical_subdir_path = os.path.join(numerical_base_dir, subdir)
            if not os.path.exists(numerical_subdir_path):
                os.makedirs(numerical_subdir_path)

            # Define the target file path in the new directory
            target_file_path = os.path.join(numerical_subdir_path, 'detection.json')

            # Copy the JSON file to the new location
            shutil.copy2(json_file_path, target_file_path)
            print(f"File {json_file_path} copied to {target_file_path}")
        else:
            print(f"Removed {json_file_path} as it contains no numerical values")
    else:
        print(f"No detection.json file found in {subdir_path}")


In [ ]:
import os

def count_directories(path):
    """Count directories in a given path."""
    return sum(1 for entry in os.listdir(path) if os.path.isdir(os.path.join(path, entry)))

# Paths to the directories
processed_json_dir = '/teamspace/studios/this_studio/processed_jsons'
numerical_detections_dir = '/teamspace/studios/this_studio/numerical_detections'

# Count directories in both paths
processed_count = count_directories(processed_json_dir)
numerical_count = count_directories(numerical_detections_dir)

# Print the counts and the difference
print(f"Total directories in processed_jsons: {processed_count}")
print(f"Total directories in numerical_detections: {numerical_count}")
print(f"Number of directories removed (no numerical data): {processed_count - numerical_count}")


In [ ]:
import os
import shutil

# Define source, bounding box, and mask directories
source_dir = '/teamspace/studios/this_studio/numerical_detections'
bounding_box_dir = '/teamspace/studios/this_studio/bounding_box'
mask_dir = '/teamspace/studios/this_studio/mask'

# Ensure the bounding box and mask directories exist
os.makedirs(bounding_box_dir, exist_ok=True)
os.makedirs(mask_dir, exist_ok=True)

# Define bounding box indices based on your provided list
bounding_box_indices = set(range(121, 166)) | set(range(156, 178)) | set(range(295, 356)) | set(range(388, 390))

# Process each subdirectory in the source directory
for subdir in os.listdir(source_dir):
    # Extract numeric part assuming format 'xmlab<number>' possibly followed by additional text
    # This regex matches numbers right after 'xmlab'
    subdir_index_str = ''.join(filter(str.isdigit, subdir[len('xmlab'):]))  # Get numeric part
    if subdir_index_str:
        subdir_index = int(subdir_index_str)  # Convert to integer
        subdir_path = os.path.join(source_dir, subdir)

        if subdir_index in bounding_box_indices:
            # Move to bounding box directory
            target_subdir_path = os.path.join(bounding_box_dir, subdir)
        else:
            # Move to mask directory
            target_subdir_path = os.path.join(mask_dir, subdir)

        # Move the folder
        shutil.move(subdir_path, target_subdir_path)
        print(f"Moved {subdir} to {target_subdir_path}")

print("Files have been organized into bounding box and mask folders.")


In [ ]:
import os

def count_folders(directory):
    """Count the number of directories within a specified directory."""
    return sum(1 for item in os.listdir(directory) if os.path.isdir(os.path.join(directory, item)))

# Define the paths to the bounding box and mask directories
bounding_box_dir = '/teamspace/studios/this_studio/bounding_box'
mask_dir = '/teamspace/studios/this_studio/mask'

# Count the directories in each
bounding_box_count = count_folders(bounding_box_dir)
mask_count = count_folders(mask_dir)

# Print the counts
print(f"Number of folders in bounding box directory: {bounding_box_count}")
print(f"Number of folders in mask directory: {mask_count}")


In [ ]:
import os

# Directory to check
bounding_box_dir = '/teamspace/studios/this_studio/bounding_box'

# List all folders and sort them numerically based on the embedded number
folders = sorted(os.listdir(bounding_box_dir), key=lambda x: int(''.join(filter(str.isdigit, x))))

# Print all folders
for folder in folders:
    print(folder)

# Optionally, count them
print(f"Total folders found: {len(folders)}")


In [ ]:
import os
import shutil

# Define the source and destination directories
source_dir = '/teamspace/studios/this_studio/processed_jsons'
destination_dir = '/teamspace/studios/this_studio/bounding_box'

# List of missing folders to move
missing_folders = ['xmlab178detection', 'xmlab356detection', 'xmlab390detection']

# Loop through the missing folders and move them if found
for folder in missing_folders:
    source_folder_path = os.path.join(source_dir, folder)
    destination_folder_path = os.path.join(destination_dir, folder)

    # Check if the folder exists in the source directory
    if os.path.exists(source_folder_path):
        # Move the folder to the bounding box directory
        shutil.move(source_folder_path, destination_folder_path)
        print(f"Moved {folder} to the bounding box directory.")
    else:
        print(f"Folder {folder} not found in the processed JSON directory.")


In [ ]:
import os
import shutil

# Define the source and destination directories
source_dir = '/teamspace/studios/this_studio/processed_jsons'
destination_dir = '/teamspace/studios/this_studio/bounding_box'

# Generate a list of missing folders based on your specifications
missing_folders = [f'xmlab{num}detection' for num in [120] + list(range(179, 200)) + list(range(391, 394))]

# Process each folder, moving it if found in the source directory
for folder_name in missing_folders:
    source_folder_path = os.path.join(source_dir, folder_name)
    destination_folder_path = os.path.join(destination_dir, folder_name)

    # Check if the folder exists in the source directory
    if os.path.exists(source_folder_path):
        # Move the folder to the bounding box directory
        shutil.move(source_folder_path, destination_folder_path)
        print(f"Moved {folder_name} to the bounding box directory.")
    else:
        print(f"Folder {folder_name} not found in the processed JSON directory.")


In [ ]:
import os

# Define the path to the bounding box directory
bounding_box_dir = '/teamspace/studios/this_studio/bounding_box'

# Define the ranges to check
ranges_to_check = list(range(295, 394)) + list(range(120, 200))  # Combines both ranges into one list

# Function to generate folder names based on the specified range
def generate_folder_name(num):
    return f"xmlab{num}detection"

# Check each folder in the defined ranges
missing_folders = []
for num in ranges_to_check:
    folder_name = generate_folder_name(num)
    folder_path = os.path.join(bounding_box_dir, folder_name)
    # Check if the folder exists
    if not os.path.exists(folder_path):
        missing_folders.append(folder_name)

# Report the findings
if missing_folders:
    print("The following folders are missing:")
    for folder in missing_folders:
        print(folder)
else:
    print("All folders from xmlab295 to xmlab393 and xmlab120 to xmlab199 are present in the bounding box directory.")


In [ ]:
import os

# Directory to check
bounding_box_dir = '/teamspace/studios/this_studio/bounding_box'

# List all folders and sort them numerically based on the embedded number
folders = sorted(os.listdir(bounding_box_dir), key=lambda x: int(''.join(filter(str.isdigit, x))))

# Print all folders
for folder in folders:
    print(folder)

# Optionally, count them
print(f"Total folders found: {len(folders)}")


In [ ]:
import os
import shutil

# Define the source and destination directories
source_dir = '/teamspace/studios/this_studio/processed_jsons'
destination_dir = '/teamspace/studios/this_studio/bounding_box'

# Generate a list of folders to move, from xmlab357detection to xmlab387detection
missing_folders = [f'xmlab{num}detection' for num in range(357, 388)]

# Process each folder, moving it if found in the source directory
for folder_name in missing_folders:
    source_folder_path = os.path.join(source_dir, folder_name)
    destination_folder_path = os.path.join(destination_dir, folder_name)

    # Check if the folder exists in the source directory
    if os.path.exists(source_folder_path):
        # Move the folder to the bounding box directory
        shutil.move(source_folder_path, destination_folder_path)
        print(f"Moved {folder_name} to the bounding box directory.")
    else:
        print(f"Folder {folder_name} not found in the processed JSON directory.")


In [ ]:
import os
import shutil

# Define the directories
bounding_box_dir = '/teamspace/studios/this_studio/bounding_box'
mask_dir = '/teamspace/studios/this_studio/mask'

# Get lists of folders in both directories
bounding_box_folders = set(os.listdir(bounding_box_dir))
mask_folders = set(os.listdir(mask_dir))

# Find intersections (folders that should not be in the mask directory)
intersecting_folders = bounding_box_folders.intersection(mask_folders)

# Remove intersecting folders from the mask directory
for folder in intersecting_folders:
    folder_path = os.path.join(mask_dir, folder)
    # Remove the folder entirely
    shutil.rmtree(folder_path)
    print(f"Removed {folder} from the mask directory as it was found in the bounding box directory.")

# Optionally, report remaining valid folders
remaining_mask_folders = set(os.listdir(mask_dir))
print("Remaining folders in the mask directory are confirmed to be exclusive from the bounding box directory.")
for folder in remaining_mask_folders:
    print(folder)


In [ ]:
import os
import zipfile

def zip_directory(folder_path, output_path):
    """Zips the contents of a folder into a zip file at the specified output path."""
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Create a proper archive path to keep the directory structure
                archive_path = os.path.relpath(os.path.join(root, file), os.path.join(folder_path, '..'))
                zipf.write(os.path.join(root, file), archive_path)
        print(f"Created zip archive: {output_path}")

# Define the directories to zip
directories_to_zip = {
    'mask': '/teamspace/studios/this_studio/mask',
    'numerical_detections': '/teamspace/studios/this_studio/numerical_detections',
    'bounding_box': '/teamspace/studios/this_studio/bounding_box'
}

# Loop through the directories and zip each one
for key, path in directories_to_zip.items():
    output_zip_path = f"/teamspace/studios/this_studio/{key}.zip"
    zip_directory(path, output_zip_path)


In [ ]:
import os
import shutil

def copy_files(src_dir, dest_dir):
    """Copy all relevant files from src_dir to dest_dir."""
    # Define relevant files
    relevant_files = ['source.jpg', 'mask.png', 'detection.json', 'question.json']
    for file_name in relevant_files:
        src_file = os.path.join(src_dir, file_name)
        dest_file = os.path.join(dest_dir, file_name)
        # Check if the source file exists before copying
        if os.path.exists(src_file):
            shutil.copy2(src_file, dest_file)
            print(f"Copied {src_file} to {dest_file}")
        else:
            print(f"File {src_file} does not exist, skipping.")

def consolidate_files(imgs_base_path, mask_base_path, bounding_box_base_path):
    """Consolidate files from imgs directory to respective mask or bounding_box directories."""
    # Loop through all subdirectories in imgs base path
    for subdir in os.listdir(imgs_base_path):
        src_dir = os.path.join(imgs_base_path, subdir)
        dest_dir = None
        # Check if the subdirectory corresponds to a mask or bounding box instance
        if subdir.endswith("detection"):
            subdir_num = int(subdir.replace("xmlab", "").replace("detection", ""))
            if subdir_num >= 120:  # Assuming all instances with 120 or above go to bounding_box
                dest_dir = os.path.join(bounding_box_base_path, subdir)
            else:  # Otherwise, they go to mask
                dest_dir = os.path.join(mask_base_path, subdir)
        
        if dest_dir and not os.path.exists(dest_dir):
            os.makedirs(dest_dir)

        # Copy relevant files
        if dest_dir:
            copy_files(src_dir, dest_dir)

# Define base paths
imgs_base_path = '/teamspace/studios/this_studio/imgs'
mask_base_path = '/teamspace/studios/this_studio/mask'
bounding_box_base_path = '/teamspace/studios/this_studio/bounding_box'

# Consolidate files
consolidate_files(imgs_base_path, mask_base_path, bounding_box_base_path)
